In [1]:
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import StandardScaler as zscore # zscore
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso #LRlasso
from joblib import dump, load #to save models in files
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import re
import json
from datetime import datetime
import os

In [36]:
with open("/home/t44p/PW_rawdata/results/newfeat_rfr/lcms/lcms_dict_nXcv.json", 'r') as json_file:
    lcms_prefix = json.load(json_file)
with open("/home/t44p/PW_rawdata/results/newfeat_rfr/gcms/gcms_dict_nXcv.json", 'r') as json_file:
    gcms_prefix = json.load(json_file)

In [39]:
gc = list(gcms_prefix.keys())
lc = list(lcms_prefix.keys())


In [46]:
print(lc)

['alpha_L_Fucose_1_Phosphate', 'Trehalose', 'Trehalose_phenolic_acid_', 'Maltotriose_', '_366_9939681_1_170199352', '_R_2_Phenylglycin', 'Oxidized_glutathione_', '_312_0943959_2_614228811', 'Guanosine_', 'gamma_Glutamylvaline_', '_512_1445532_3_132648409', 'Catechol_', '_303_0720727_3_335904942', 'Quinic_acid_derivative_3_56', '_374_1568694_3_674168475', 'Phaseoloidin', '_496_1502126_3_792994891', 'Vanilloloside', 'CGA_hexose_', 'Phenolic_glycoside_', 'gamma_Glutamylisoleucine_', 'L_gamma_glutamyl_L_isoleucine', '_658_1575839_4_528042834', 'Phenylalanylglutamic_acid_', '_5_Chlorogenic_acid', '_523_1663837_5_238225973', '_487_2029578_5_338775921', 'Quinic_acid_161_05_', 'Quinic_acid_derivative_with_he', 'Chlorogenic_acid_', 'Coumaroyl_quinic_acid_', '_351_1294837_5_809380415', 'Quercetin_glucose_glucose_rham', '_427_1824019_5_95992104', 'Quercetin_glc_glc_rha', '_567_2828405_6_133036019', 'Quercetin_3_7_di_glucoside_', '_441_1978503_6_218123415', '_4_O_p_Coumaroylquinic_acid_', 'Quercet

In [19]:
lasso_df = pd.read_csv("/home/t44p/PW_rawdata/results/newfeat_lasso/newfeat_lasso_performance.csv", sep="\t")
lasso_df = lasso_df[lasso_df['predictive_power'] == 'good'].copy()
lasso_good = list(lasso_df['target'])


In [22]:
rfr_df = pd.read_csv("/home/t44p/PW_rawdata/results/newfeat_rfr/newfeat_rfr_performance.csv", sep="\t")
rfr_df = rfr_df[rfr_df['predictive_power'] == 'good'].copy()
rfr_good = list(rfr_df['target'])


In [31]:
shared = set(rfr_good).intersection(set(lasso_good)) # shared by both, good predictors independent of approach
uniq_lasso = set(lasso_good).difference(set(rfr_good)) # uniq to lasso
uniq_rfr = set(rfr_good).difference(set(lasso_good)) # uniq to rfr

In [47]:
print(uniq_rfr)

{'Cellobiose_361_204_rt14_40_nXcv', 'Caffeic_acid_glucoside_L_nXcv', 'Conduritol_B_Epoxide_L_nXcv', 'Isorhamnetin_3_O_rutinoside_nXcv', '_641_171822_6_885700966_nXcv', '_487_2029578_5_338775921_nXcv', '_6_phospho_gluconate_nXcv', 'Fructose_1_6_biosphasphate_L_nXcv', 'Quinic_acid_161_05__nXcv', 'Kaempferol_glucose_xylose_L_nXcv'}


# funktioniert ne


In [49]:
import pandas as pd

# Function to remove '_nXcv' suffix and categorize entries
def process_entries(entries, prefixes):
    processed = []
    for entry in entries:
        # Remove the suffix
        entry_processed = entry.split('_nXcv')[0]
        # Check if the entry belongs to the given prefixes
        if any(entry_processed.startswith(prefix) for prefix in prefixes):
            processed.append(entry_processed)
    return processed

# Process entries for each category
shared_processed = process_entries(shared, gc + lc)
uniq_lasso_processed = process_entries(uniq_lasso, gc + lc)
uniq_rfr_processed = process_entries(uniq_rfr, gc + lc)

# Create empty tables for lc and gc
lc_table = pd.DataFrame(columns=['shared', 'uniq_lasso', 'uniq_rfr'])
gc_table = pd.DataFrame(columns=['shared', 'uniq_lasso', 'uniq_rfr'])

# Function to add a row to the table
def add_row(table, entry, category):
    row = {'shared': None, 'uniq_lasso': None, 'uniq_rfr': None}
    row[category] = entry
    return pd.concat([table, pd.DataFrame([row])], ignore_index=True)

# Populate the tables
for entry in shared_processed + uniq_lasso_processed + uniq_rfr_processed:
    if any(entry.startswith(prefix) for prefix in lc):
        category = 'shared' if entry in shared_processed else 'uniq_lasso' if entry in uniq_lasso_processed else 'uniq_rfr'
        lc_table = add_row(lc_table, entry, category)
    elif any(entry.startswith(prefix) for prefix in gc):
        category = 'shared' if entry in shared_processed else 'uniq_lasso' if entry in uniq_lasso_processed else 'uniq_rfr'
        gc_table = add_row(gc_table, entry, category)

# Printing the tables
print("LC Table:")
print(lc_table)
print("\nGC Table:")
print(gc_table)


LC Table:
                            shared                     uniq_lasso  \
0                   Saponin_10_183                           None   
1                        Inosine_L                           None   
2            Oxidized_glutathione_                           None   
3     Phenylalanyaspartatic_acid_L                           None   
4                Quercetin_glucose                           None   
5          _1_O_Feruloyl_glucose_L                           None   
6          Feruloyl_di_glucoside_L                           None   
7                     Saponin_9_88                           None   
8   Quercitin_3_7_diglucoside_6_rh                           None   
9     Isorhamnetin_3_O_glucoside_L                           None   
10      p_coumaroyl_di_glucoside_L                           None   
11  Nicotinamide_adenine_dinucleot                           None   
12         Caffeoylshikimic_acid_L                           None   
13  Kaempferol_3_O_rhamn